Pitch, duration in beats, hold last note


We divide up the the piece to discrete time segments, and each time segment, we check 

(Tempo is a pre-determined constant)


Every timestep (probably 8th notes), we have:
- Each timestep = vector of ~130 floats
- A pitch (or rest or hold) --- (We could do relative pitch, but lets put that off until later)

"One Hot Encoding" - We've got a discrete variable, so we want to break them up into a sequence of probabilities



In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(123)

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils

import music21 as m21

In [14]:
song = music21.converter.parse('data/bach-chorales/bach-1.mid')
song.analyze('key')
# song.show('midi')

song.measures(0, 1)

<music21.stream.Score 0x1112844e0>

In [15]:
song.show('text')


{0.0} <music21.stream.Part 0x11121e550>
    {0.0} <music21.tempo.MetronomeMark animato Quarter=120.0>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.stream.Voice 0x11127dda0>
        {0.0} <music21.chord.Chord D3 A3 F#4 D5>
        {2.0} <music21.chord.Chord F#3 D4 A4 A4>
        {4.0} <music21.chord.Chord G#3 B4>
        {5.0} <music21.chord.Chord F#3 C#5>
        {6.0} <music21.chord.Chord E3 G#3>
        {7.0} <music21.chord.Chord F#3 A3>
        {8.0} <music21.chord.Chord G#3 B3 E4 D5>
        {10.0} <music21.chord.Chord A3 C#5>
        {12.0} <music21.chord.Chord B-3 C#4 F#4 C#5>
        {14.0} <music21.note.Note B>
        {15.0} <music21.note.Note F#>
        {16.0} <music21.note.Note G>
        {17.0} <music21.chord.Chord D3 F#4>
        {18.0} <music21.chord.Chord E3 G4 B4>
        {20.0} <music21.chord.Chord F#3 B-4>
        {22.0} <music21.chord.Chord B2 D4 F#4 B4>
        {24.0} <music21.chord.Chord B3 D4 F#4 B4>
        {26.0} <music21.chord.Chord B3 D5 F#5

In [16]:
for x in song.notes: #.notesAndrests
    x

In [17]:
song.voices
song.parts[0].sorted.voices[3]#elements#.show('text')

<music21.stream.Voice 0x1114f56a0>

In [18]:
song.analyze('ambitus')
song.flat.notes[0].pitches
song.flat.notes[1].pitches
song.flat.notes[9].__dict__
song.flat.notes[14]
# get_part_1(song).notes.show('text')
get_part_1(song).duration.quarterLength

208.0

In [51]:
# Here's some actual useful code:

# A0 is 21 and C8 is 108
HOLD_NOTE = 20
SILENCE = 19

# Gets one of the voices in the song
# - allows us to extract the 'melody' and 'harmony'
def get_part_1(song):
    return song.parts[0].sorted.voices[0]
def get_part_2(song):
    return song.parts[0].sorted.voices[1]

# Converts note or chord to a midi pitch (int)
def pitch(note):
    try:
        #... Should probably change this part
        return note.pitches[0].midi # If it's a chord, get top note
    except AttributeError:
        return note.pitch.midi # if it's not a chord


# music21.Stream -> [int...]
def get_pitch_array(song):
    # Divisions are in quarter notes... we may want to change this
    array = []
    current_note = 0
    for time in range(0, int(song.duration.quarterLength)):
        # Figure out current note
        note = song.sorted.notes[current_note]
        if note.offset + note.duration.quarterLength <= time:
            current_note += 1
            try:
                note = song.sorted.notes[current_note+1]
            except IndexError: # last note is longer than a quarter note
                current_note += -1

        # Add current note to the array
        if note.offset > time:
            array.append(SILENCE)
        elif note.offset > time-1: # Note started in the last quarter note
            array.append(pitch(note))
        else:
            array.append(HOLD_NOTE)
    return array

# Converts pitch number to a tuple with "one hot encoding"
LOWEST_PITCH=SILENCE
HIGHEST_PITCH=108 # C8
def pitch_to_tuple_vector(pitch):
    list = [0] * (HIGHEST_PITCH-LOWEST_PITCH)
    list[pitch-LOWEST_PITCH] = 1
    return tuple(list)

def tuple_vector_to_pitch(tuple):
    index = max(enumerate(tuple), key=lambda x: x[1])[0]
    return LOWEST_PITCH + index

def pitch_array_to_stream(pitch_array):
    stream = music21.stream.Stream()
    for pitch in pitch_array:
        if pitch == HOLD_NOTE:
            stream[-1].quarterLength += 1
        elif pitch != SILENCE:
            stream.append(music21.note.Note(pitch))
    return stream

In [52]:
# Here's how we get our training data:
melody_vectors = list(map(pitch_to_tuple_vector, get_pitch_array(get_part_1(song))))
harmony_vectors= list(map(pitch_to_tuple_vector, get_pitch_array(get_part_2(song))))

# Converting back:
harmony_pitch_array = list(map(tuple_vector_to_pitch, melody_vectors))
pitch_array_to_stream(harmony_pitch_array).show('midi')

# There's a problem somewhere in here... if I run:
pitch_array_to_stream(get_pitch_array(get_part_2(song))).show('midi')
# I get a different result than in the Converting back section
